3. Go through image classification and break down code
4. Go through regression and break down code
5. Complete subclass section
6. Complete saving and restoring model section
7. Go through and break down subclass
8. Go through tensorboard
9. Go through fine tuning as a medium article start
10. Place on github

In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

In [3]:
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
x_train, x_validation, y_train, y_validation = train_test_split(x_train_full, y_train_full)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_validation = scaler.transform(x_validation)
x_test = scaler.transform(x_test)

In [4]:
# Building a model using the sequential api
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=x_train.shape[1:]),
    keras.layers.Dense(1)
])

In [6]:
# Building complex models using the functional api (wide and deep networks)(Functional API)

input_ = keras.layers.Input(shape=x_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [5]:
# Buiding a network with two inputs using functional api

input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])



In [7]:
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(x_train, y_train, epochs=30, validation_data=(x_validation, y_validation))
mse_test = model.evaluate(x_test, y_test)
X_new = x_test[:3]
y_pred = model.predict(X_new)
y_pred

Train on 11610 samples, validate on 3870 samples
Epoch 1/30
11610/11610 [==============================] - 3s 229us/sample - loss: 0.9511 - val_loss: 111.6381
Epoch 2/30
11610/11610 [==============================] - 1s 78us/sample - loss: 3.0317 - val_loss: 631.5433
Epoch 3/30
11610/11610 [==============================] - 1s 93us/sample - loss: 7.0851 - val_loss: 138.3526
Epoch 4/30
11610/11610 [==============================] - 1s 96us/sample - loss: 1.8839 - val_loss: 37.0720
Epoch 5/30
11610/11610 [==============================] - 1s 87us/sample - loss: 0.8154 - val_loss: 9.3305
Epoch 6/30
11610/11610 [==============================] - 1s 83us/sample - loss: 3.1454 - val_loss: 2.3460
Epoch 7/30
11610/11610 [==============================] - 1s 87us/sample - loss: 0.4067 - val_loss: 0.8037
Epoch 8/30
11610/11610 [==============================] - 1s 88us/sample - loss: 0.4955 - val_loss: 0.4763
Epoch 9/30
11610/11610 [==============================] - 1s 92us/sample - loss: 0.3978

array([[1.447826  ],
       [1.33487   ],
       [0.99885243]], dtype=float32)

In [ ]:
# With a model with two input it is require a pair of matrices for the training data, one per input, this rule also applies to validation and test data when calling the evaluate or predict function

model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=1e-3))

x_train_A, x_train_B = x_train[:, :5], x_train[:, 2]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2]
x_test_A, x_test_B = x_test[:, :5], x_test[:, 2]
X_new_A, X_new_B = x_test_A[:3], x_test_B[:3]

history = model.fit((x_train_A, x_train_B), y_train, epochs=20, validation_data=((x_valid_A, x_valid_B), y_valid))
mse_test = model.evaluate((x_test_A, x_test_B), y_test)
y_pred = model.predict((x_new_A, x_new_B))

There are variety of use cases where networks with multiple ouputs are useful

- Location and Classification involved in object detection tasks. The location is a regression task as we have to find the corronidate of the objects center as well as the width and the height, while performing classification

- You may have multiple independednt tasks based on the same data. Sure you could train one neural network per task, but in many cases you will get better results on all taks by training a single neural network with one output per task. This is because the neural network can learn features in the data that are usefull across tasks. For example you could perform multitask classificatio on pictures of faces, using one output to classificy the person's facial expression and another to identify whether they are wearing glasses or not

- Another use case is a regularization technique(a training contraints whose objective is to reduce overfitting, thus improving the models ability to generalize). For example you may want ot add some auzilary outputs in a neural network architecture to ensure that the eunderlying part of the netowrk learns somehting useful on its own, without relying on the rest of the network

In [ ]:
# Creating a network with an output and an auxilary output and also 2 input layers

input_a = keras.layers.Input(shape=[5], name='wide_input')
input_b = keras.layers.Input(shape=[6], name='deep_input')
hidden_1 = keras.layers.Dense(100, activation='relu')(input_b)
hidden_2 = keras.layers.Dense(60, activation='relu')(input_a)
concat = keras.layers.Concatenate()([input_a, hidden_2])
output = keras.layers.Dense(1, name='main_output')(concat)
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)
model = keras.Model(inputs=[input_a, input_b], outputs=[output, aux_output])


# Each output will have its own loss function
model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1], optimizer='sgd')
history = model.fit(
    [x_train_A, x_train_B], 
    [y_train, y_train], 
    epochs=20, validation_data=([x_valid_A, x_valid_B], [y_valid, y_valid]))

total_loss, main_loss, aux_loss = model.evaluate([x_test_A, x_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([x_new_A, x_new_b])

In [ ]:
# Createing a model by subclassing the Model Class

class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_a, input_b = inputs
        hidden1 = self.hidden1(input_a)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_a, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel()